In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

file_path = 'genres_v2.csv'
df = pd.read_csv(file_path)
df = df.dropna(subset=['song_name', 'genre', 'tempo', 'valence', 'liveness', 'instrumentalness'])
# Tydligen ungefär 20k värt av sånger med NaN vid sångnamnet
columns_to_remove = ['Unnamed: 0', 'title', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature','type']
df = df.drop(columns=columns_to_remove)
# df = df.dropna()
df


<ipython-input-1-bac067ff55e1>:6: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre,song_name
0,0.831,0.814,2,-7.364,1,0.4200,0.05980,0.013400,0.0556,0.3890,156.985,Dark Trap,Mercury: Retrograde
1,0.719,0.493,8,-7.230,1,0.0794,0.40100,0.000000,0.1180,0.1240,115.080,Dark Trap,Pathology
2,0.850,0.893,5,-4.783,1,0.0623,0.01380,0.000004,0.3720,0.0391,218.050,Dark Trap,Symbiote
3,0.476,0.781,0,-4.710,1,0.1030,0.02370,0.000000,0.1140,0.1750,186.948,Dark Trap,ProductOfDrugs (Prod. The Virus and Antidote)
4,0.798,0.624,2,-7.668,1,0.2930,0.21700,0.000000,0.1660,0.5910,147.988,Dark Trap,Venom
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,0.538,0.819,1,-6.780,1,0.4670,0.00854,0.000000,0.1310,0.2520,160.299,Hiphop,Daddy's Girl
21521,0.415,0.548,7,-8.563,1,0.2850,0.22400,0.000000,0.1220,0.5280,154.272,Hiphop,'94
21522,0.785,0.648,7,-6.907,1,0.2790,0.01570,0.001980,0.1000,0.5060,184.160,Hiphop,Born 2 Live
21523,0.747,0.832,3,-5.737,0,0.1850,0.23300,0.000014,0.1300,0.7330,188.760,Hiphop,Can't Hold On


https://www.kaggle.com/datasets/mrmorj/dataset-of-songs-in-spotify/


In [2]:
unique_values = df['valence'].unique()
unique_values
# För att kolla diverse kolumners values att där inte var något som kan kasta den helt i knut

array([0.389 , 0.124 , 0.0391, ..., 0.0899, 0.0897, 0.0828])

In [3]:
# För att göra genren till numeriska vektorer
tfidf_v = TfidfVectorizer(stop_words='english')
tfidf_m = tfidf_v.fit_transform(df['genre'])

# Räknar "cosine similiraty" mellan songerna baserat på deras tfidf vektorer
cosine_sim = linear_kernel(tfidf_m, tfidf_m)

# bara baserat på genre liknelse
def get_rec(name, top_songs_amount, cosine_sim):
  if name not in df['song_name'].values:
    return 'Song not found' # Snabb koll att sången finns så de inte behöver komma error i onödan om man t.ex. stavar fel

  idx = df.loc[df['song_name'] == name].index[0]
  sim_scores = list(enumerate(cosine_sim[idx]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  sim_scores = sim_scores[1:top_songs_amount+1] # Ger top n sånger enligt vad man säger att den ska ge
  song_indices = [i[0] for i in sim_scores]
  return df['song_name'].iloc[song_indices]

user_input = 'Venom'
recommendations = get_rec(user_input, 10, cosine_sim) #Denna baserar sig endast på genren
recommendations

1                                         Pathology
2                                          Symbiote
3     ProductOfDrugs (Prod. The Virus and Antidote)
4                                             Venom
5                                           Gatteka
6                                kamikaze (+ pulse)
7               T.R.U. (Totally Rotten Underground)
8                      I Put My Dick in Your Mental
9                                         Andromeda
10                                        BRAINFOOD
Name: song_name, dtype: object

In [4]:
from sklearn.metrics.pairwise import cosine_similarity


features = ['danceability', 'energy', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'] # Använder alla dessa "features" som används i rekommendations systemet.

for feature in features:
  df[feature]  = (df[feature] - df[feature].min()) / (df[feature].max() - df[feature].min()) # normalize

# Cosine similarity räkning igen, behöver ingen tdidf vektorisering när de färdigt är numeriska värden (normalisera bara dem ovanför)
cosine_sim2 = cosine_similarity(df[features])

recommendations2 = get_rec('Venom', 10, cosine_sim2) # Ger cosine similarity som en parameter så behöver jag inte skriva funktionen pånytt
recommendations2


5800                                Venom
11285                               Venom
8281              Venom (feat. Shakewell)
9687                             Meet God
8118                              Memphis
1085                          Speak on It
11183                             WARTIME
5419     Walk Em Down (feat. Roddy Ricch)
9860     Walk Em Down (feat. Roddy Ricch)
4996                           Whiplash'd
Name: song_name, dtype: object

 # Hybrid:


In [5]:
nan_locations = df.isna()

print(nan_locations)

       danceability  energy    key  loudness   mode  speechiness  \
0             False   False  False     False  False        False   
1             False   False  False     False  False        False   
2             False   False  False     False  False        False   
3             False   False  False     False  False        False   
4             False   False  False     False  False        False   
...             ...     ...    ...       ...    ...          ...   
21520         False   False  False     False  False        False   
21521         False   False  False     False  False        False   
21522         False   False  False     False  False        False   
21523         False   False  False     False  False        False   
21524         False   False  False     False  False        False   

       acousticness  instrumentalness  liveness  valence  tempo  genre  \
0             False             False     False    False  False  False   
1             False             Fal

In [4]:
import math
import random
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

# Hade såpass mycke issues att tog helt från noll


# Dataset osv
df = pd.read_csv('genres_v2.csv')
scaler = StandardScaler()
df = df.apply(pd.to_numeric, errors='coerce')


# Hantera NaN (Varit ett massivt problem) och extrema värden med att byta dem till 0
df = df.dropna(subset=['song_name']) # Fanns såpass många med NaN i namnet att det måstes ta bort, det crashade x antal gånger för att RAM tog slut
df.fillna(0, inplace=True)
df = df.apply(lambda x: np.where(np.abs(x) > 1e10, 0, x))  # Replace extremely large values with 0

features = ['danceability', 'energy', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
for feature in features:
  df[feature]  = (df[feature] - df[feature].min()) / (df[feature].max() - df[feature].min()) # normalisera


# CBF kalkyleringen
# Cosine similarity räkning igen, behöver ingen tdidf vektorisering när de färdigt är numeriska värden (normalisera bara dem ovanför)
cbf_sim = cosine_similarity(df[features])



# CF kalkyleringen
user_interaction = [random.choice(range(len(df))) for _ in range(5)]  # Simulera 5 stycker användare interaktioner
cf_sim = [[int(song_name in user_interaction) for song_name in df['song_name']]]

# Hybrida parametrarna
alpha = 0.6  # vikten för CBF. 1 - alpha är CF vikten

# Kombinera båda
combined_sim = alpha * cbf_sim + (1 - alpha) * np.array(cf_sim)

# Användar rekommendationer
user_id = 'user1'
user_rating = []

cbf_predictions = []

for song_name in range(len(df)):
    if song_name not in user_interaction:
        # Kalkyleringen av hybrida "Scoren"
        hybrid_score = combined_sim[song_name, song_name]

        cbf_predictions.append((song_name, hybrid_score))

cbf_predictions.sort(key=lambda x: x[1], reverse=True)

top_recommendations = cbf_predictions[:3]

print("Top Recommendations:")
for song_idx, similarity in top_recommendations:
    print(f"Song {song_idx}: Hybrid Score = {similarity}")

print(df.iloc[user_interaction])


<ipython-input-4-b8385f842d47>:12: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('genres_v2.csv')


Top Recommendations:
Song 3: Hybrid Score = 1.0
Song 98: Hybrid Score = 1.0
Song 38: Hybrid Score = 0.9999999999999998
       danceability    energy       key  loudness  mode  speechiness  \
2354       0.826580  0.514357  0.363636  0.636166     0     0.015922   
4934       0.636953  0.208489  0.181818  0.285019     1     0.318103   
19665      0.735818  0.161049  0.272727  0.340446     0     0.058844   
12169      0.181524  0.810237  0.636364  0.923619     1     0.980962   
5703       0.881686  0.445693  0.454545  0.471037     1     0.015403   

       acousticness  instrumentalness  liveness   valence  ...   id  uri  \
2354       0.283622          0.000000  0.097519  0.348667  ...  0.0  0.0   
4934       0.072257          0.488323  0.080669  0.039015  ...  0.0  0.0   
19665      0.360139          0.003439  0.100981  0.529235  ...  0.0  0.0   
12169      0.002759          0.000000  0.240623  0.192820  ...  0.0  0.0   
5703       0.026125          0.016985  0.020081  0.938736  ...  0.0 